In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import seaborn as sns
import os

# --- Etapa 0: Configuração ---
# Criar diretório para salvar os gráficos, se não existir
output_dir = '/kaggle/working/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- Etapa 1: Carregamento e Pré-processamento dos Dados ---

# Carregar o dataset
try:
    df = pd.read_csv('/kaggle/input/dataset-clean/dados_limpos.csv')
    print("Dataset carregado do caminho Kaggle.")
except FileNotFoundError:
    try:
        df = pd.read_csv('dados_limpos.csv') 
        print("Dataset carregado localmente.")
    except FileNotFoundError:
        print("Arquivo 'dados_limpos.csv' não encontrado no caminho Kaggle nem localmente.")
        print("Por favor, certifique-se de que o arquivo está no diretório correto ou ajuste o caminho.")
        exit()

# Selecionar colunas relevantes
colunas_features = ['Nível de ensino alcançado', 'Tempo de experiência na área de dados']
coluna_target = 'Faixa salarial mensal'
colunas_necessarias = colunas_features + [coluna_target]

# Remover linhas com valores ausentes nas colunas cruciais
df_limpo = df[colunas_necessarias].copy()
df_limpo.dropna(subset=colunas_necessarias, inplace=True)

# --- Etapa 2: Engenharia de Features e Criação da Variável Alvo ---

# Mapeamento ordinal para 'Nível de ensino alcançado'
nivel_ensino_map = {
    'Estudante de Graduação': 0,
    'Graduação/Bacharelado': 1,
    'Pós-graduação': 2,
    'Mestrado': 3,
    'Doutorado ou Phd': 4
}
df_limpo['formacao_academica_encoded'] = df_limpo['Nível de ensino alcançado'].map(nivel_ensino_map)

# Mapeamento ordinal para 'Tempo de experiência na área de dados'
experiencia_map = {
    'Menos de 1 ano': 0,
    'de 1 a 2 anos': 1,
    'de 3 a 4 anos': 2,
    'de 4 a 6 anos': 3,
    'de 5 a 6 anos': 3, 
    'de 7 a 10 anos': 4,
    'Mais de 10 anos': 5
}
df_limpo['experiencia_profissional_encoded'] = df_limpo['Tempo de experiência na área de dados'].map(experiencia_map)


# Mapeamento ordinal para 'Faixa salarial mensal' para criar a variável alvo binária
salario_map_ordinal = {
    'Menos de R$ 1.000/mês': 0,
    'de R$ 1.001/mês a R$ 2.000/mês': 1,
    'de R$ 2.001/mês a R$ 3.000/mês': 2,
    'de R$ 3.001/mês a R$ 4.000/mês': 3,
    'de R$ 4.001/mês a R$ 6.000/mês': 4,
    'de R$ 6.001/mês a R$ 8.000/mês': 5,
    'de R$ 8.001/mês a R$ 12.000/mês': 6,
    'de R$ 12.001/mês a R$ 16.000/mês': 7,
    'de R$ 16.001/mês a R$ 20.000/mês': 8,
    'de R$ 20.001/mês a R$ 25.000/mês': 9,
    'de R$ 25.001/mês a R$ 30.000/mês': 10,
    'de R$ 30.001/mês a R$ 40.000/mês': 11,
    'Acima de R$ 40.001/mês': 12
}
df_limpo['faixa_salarial_encoded'] = df_limpo['Faixa salarial mensal'].map(salario_map_ordinal)

# Remover NaNs que podem surgir de mapeamentos incompletos
df_limpo.dropna(subset=['formacao_academica_encoded',
                         'experiencia_profissional_encoded',
                         'faixa_salarial_encoded'], inplace=True)

# Criar variável alvo binária: 0 para salários até R$ 8.000 (<=5), 1 para salários acima (>5)
df_limpo['salario_alto'] = df_limpo['faixa_salarial_encoded'].apply(lambda x: 1 if x > 5 else 0)

# Preparar Features (X) e Target (y)
X = df_limpo[['formacao_academica_encoded', 'experiencia_profissional_encoded']]
y = df_limpo['salario_alto']

# Verificar se há dados suficientes
if X.shape[0] < 10 or len(y.unique()) < 2:
    print("Não há dados suficientes ou classes suficientes após o pré-processamento para treinar o modelo.")
    print(f"Tamanho de X: {X.shape}, Classes em y: {y.unique()}")
    exit()

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# --- Etapa 3: Desenvolvimento do Modelo de Machine Learning - Random Forest ---

# Definir a grade de parâmetros para o GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 7],
    'class_weight': ['balanced_subsample', 'balanced']
}

# Instanciar o RandomForestClassifier
rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)

# Instanciar o GridSearchCV
# cv=3 para ser mais rápido, idealmente cv=5 ou cv=10
grid_search = GridSearchCV(estimator=rf_base, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=1, scoring='accuracy')

print("Iniciando a busca de hiperparâmetros com GridSearchCV...")
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado pelo GridSearchCV
best_rf_model = grid_search.best_estimator_

print("\nMelhores Parâmetros Encontrados pelo GridSearchCV:")
print(grid_search.best_params_)

# Previsões com o melhor modelo
y_pred_train = best_rf_model.predict(X_train)
y_pred_test = best_rf_model.predict(X_test)
y_pred_proba_test = best_rf_model.predict_proba(X_test)[:, 1] # Probabilidades para ROC

# --- Etapa 4: Avaliação do Modelo ---
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

print(f"\nAcurácia do Modelo no Conjunto de Treinamento: {accuracy_train:.4f}")
print(f"Acurácia do Modelo no Conjunto de Teste: {accuracy_test:.4f}")
print(f"Diferença de Acurácia (Treino - Teste): {accuracy_train - accuracy_test:.4f}\n")

print("Relatório de Classificação no Conjunto de Teste:")
print(classification_report(y_test, y_pred_test, target_names=['Salário Baixo/Médio', 'Salário Alto']))

print("\nParâmetros do Melhor Modelo Random Forest Utilizado:")
print(best_rf_model.get_params())

# --- Etapa 5: Geração de Gráficos ---

# 5.1. Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Salário Baixo/Médio', 'Salário Alto'],
            yticklabels=['Salário Baixo/Médio', 'Salário Alto'])
plt.title('Matriz de Confusão (Conjunto de Teste)')
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.savefig(os.path.join(output_dir, 'matriz_confusao.png'), bbox_inches='tight')
# plt.show() # Comentado para evitar interrupção se rodando em script não interativo
plt.close() 

# 5.2. Exemplo de uma Árvore de Decisão da Floresta



plt.figure(figsize=(40, 20)) 
plot_tree(best_rf_model.estimators_[0],
          feature_names=['Nível de Formação (Encoded)', 'Tempo de Experiência (Encoded)'],
          class_names=['Salário Baixo/Médio', 'Salário Alto'],
          filled=True,
          rounded=True,
          impurity=True,  # Mostra a impureza (Gini)
          proportion=False, # Mostra contagens absolutas em 'value'
          fontsize=7,  # Reduzido de 8 para 7 - ajuste conforme necessário
          max_depth=4) # Mantém a profundidade limitada para visualização
plt.title('Exemplo de uma Árvore de Decisão do Random Forest (Profundidade limitada para visualização)', fontsize=20) # Aumenta o tamanho da fonte do título
plt.savefig(os.path.join(output_dir, 'arvore_decisao_exemplo.png'), bbox_inches='tight', dpi=300) # Adiciona dpi para melhor resolução
# plt.show() # Comente/descomente conforme sua necessidade de visualização interativa
plt.close() 

# 5.3. Importância das Features
importances = best_rf_model.feature_importances_
feature_names_importances = ['Nível de Formação', 'Tempo de Experiência']
forest_importances = pd.Series(importances, index=feature_names_importances)

fig, ax = plt.subplots(figsize=(10,6))
forest_importances.sort_values(ascending=False).plot.bar(ax=ax)
ax.set_title("Importância das Features (Random Forest)")
ax.set_ylabel("Redução média de impureza (Gini)")
fig.tight_layout()
plt.savefig(os.path.join(output_dir, 'importancia_features.png'), bbox_inches='tight')
# plt.show()
plt.close()

# 5.4. Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_test)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC (Receiver Operating Characteristic)')
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'curva_roc.png'), bbox_inches='tight')
# plt.show()
plt.close()

# 5.5. Gráfico de Distribuição da Variável Alvo (Salário Alto)
plt.figure(figsize=(7, 5))
sns.countplot(x='salario_alto', data=df_limpo, palette='viridis')
plt.title('Distribuição da Variável Alvo (0: Salário Baixo/Médio, 1: Salário Alto)')
plt.xlabel('Categoria Salarial')
plt.ylabel('Contagem')
plt.xticks([0,1], ['Salário Baixo/Médio (<= R$8k)', 'Salário Alto (> R$8k)'])
plt.savefig(os.path.join(output_dir, 'distribuicao_target.png'), bbox_inches='tight')
# plt.show()
plt.close()

# 5.6. Análise: Nível de Ensino vs. Proporção de Salário Alto
nivel_ensino_map_inv = {v: k for k, v in nivel_ensino_map.items()}
df_limpo['Nivel de ensino (labels)'] = df_limpo['formacao_academica_encoded'].map(nivel_ensino_map_inv)

plt.figure(figsize=(12, 7))
sns.barplot(
    x='Nivel de ensino (labels)',
    y='salario_alto',
    data=df_limpo,
    estimator=lambda x: sum(x==1)*100.0/len(x) if len(x) > 0 else 0.0,
    palette='coolwarm',
    order=list(nivel_ensino_map.keys()) # Garante a ordem correta das categorias
)
plt.title('Proporção de Profissionais com Salário Alto por Nível de Ensino')
plt.xlabel('Nível de Ensino Alcançado')
plt.ylabel('Proporção de Salário Alto (%)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'formacao_vs_salario_alto.png'), bbox_inches='tight')
# plt.show()
plt.close()

# 5.7. Análise: Tempo de Experiência vs. Proporção de Salário Alto
experiencia_map_inv = {v: k for k, v in experiencia_map.items()}
ordered_experiencia_labels = sorted(experiencia_map, key=experiencia_map.get)
unique_ordered_experiencia_labels = []
seen_values = set()
for label in ordered_experiencia_labels:
    encoded_val = experiencia_map[label]
    if encoded_val not in seen_values:
        original_label_for_unique_value = next(k for k,v in experiencia_map.items() if v == encoded_val and k in ordered_experiencia_labels)
        if original_label_for_unique_value not in unique_ordered_experiencia_labels: # Para evitar duplicatas de labels se vários mapeiam para o mesmo valor encodado mas queremos apenas um representante
             unique_ordered_experiencia_labels.append(original_label_for_unique_value)
        seen_values.add(encoded_val)


df_limpo['Experiencia (labels)'] = df_limpo['experiencia_profissional_encoded'].map(experiencia_map_inv)

valid_experiencia_labels_in_df = df_limpo['Experiencia (labels)'].dropna().unique()
final_ordered_experiencia_labels = [lbl for lbl in unique_ordered_experiencia_labels if lbl in valid_experiencia_labels_in_df]


plt.figure(figsize=(12, 7))
sns.barplot(
    x='Experiencia (labels)',
    y='salario_alto',
    data=df_limpo,
    estimator=lambda x: sum(x==1)*100.0/len(x) if len(x) > 0 else 0.0,
    palette='coolwarm',
    order=final_ordered_experiencia_labels 
)
plt.title('Proporção de Profissionais com Salário Alto por Tempo de Experiência')
plt.xlabel('Tempo de Experiência na Área de Dados')
plt.ylabel('Proporção de Salário Alto (%)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'experiencia_vs_salario_alto.png'), bbox_inches='tight')
# plt.show()
plt.close()

print(f"\nTodos os gráficos foram salvos no diretório: {output_dir}")


Dataset carregado do caminho Kaggle.
Iniciando a busca de hiperparâmetros com GridSearchCV...
Fitting 3 folds for each of 162 candidates, totalling 486 fits

Melhores Parâmetros Encontrados pelo GridSearchCV:
{'class_weight': 'balanced_subsample', 'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 100}

Acurácia do Modelo no Conjunto de Treinamento: 0.7544
Acurácia do Modelo no Conjunto de Teste: 0.7283
Diferença de Acurácia (Treino - Teste): 0.0262

Relatório de Classificação no Conjunto de Teste:
                     precision    recall  f1-score   support

Salário Baixo/Médio       0.84      0.65      0.73       568
       Salário Alto       0.64      0.84      0.72       422

           accuracy                           0.73       990
          macro avg       0.74      0.74      0.73       990
       weighted avg       0.76      0.73      0.73       990


Parâmetros do Melhor Modelo Random Forest Utilizado:
{'bootstrap': True, 'ccp_alpha': 0.0, 'cla